In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as smp
from scipy.integrate import odeint
import plotly.graph_objects as go
from IPython.display import HTML
from sympy.matrices import Matrix
from sympy.vector import Vector
import plotly.express as px
from ipywidgets import interact
from ipywidgets import interactive
from ipywidgets import widgets
from scipy.integrate import odeint
import plotly.graph_objects as go
from IPython.display import HTML
from scipy.integrate import solve_ivp
import plotly.io as pio

### Free Energy Density
We start by the free energy density in terms of unit magnetizations $\textbf{m}_{1}$ and $\textbf{m}_{2}$:

$$ \frac{\epsilon}{\hbar\gamma_{0}} =H_{ex} \textbf{m}_{1}\cdot \textbf{m}_{2}-K_{z}[(\textbf{e}_{h}\cdot \textbf{m}_{1})^2 +(\textbf{e}_{h}\cdot \textbf{m}_{2})^2] - D\textbf{e}_{D}\cdot(\textbf{m}_{1}\times\textbf{m}_{2})-H\cdot(\textbf{m}_{1}+\textbf{m}_{2}) $$

### LLG Equation
Dynamics are described by two coupled LLG equation:

$$\frac{d\vec{m}_{1,2}}{dt}= \gamma_{0} (\vec{\textbf{m}}_{1,2} \times H^{eff}_{1,2})+ \alpha_{0} \vec{\textbf{m}}_{1,2} \times\frac{d\vec{\textbf{m}}_{1,2}}{dt} +\gamma_{0}[H_{FL}+H_{Oe}] \hat{y} \times \vec{\textbf{m}}_{1,2}+\gamma_{0}H_{DL}\vec{\textbf{m}}_{1,2}\times[\hat{y}\times \vec{\textbf{m}}_{1,2}] $$

### Effective Field
Where the effective fields for each subllattice $H^{eff}_{1,2}$ is given by:

$$H^{eff}_{1,2} =\frac{\delta\epsilon}{\hbar\gamma_{0}\delta\vec{\textbf{m}}_{1,2} } $$

We define the necessary variables for symbolic calculation using Sympy:

In [2]:
t = smp.symbols('t', real=True)
Hx, Hy, Hz, a, g, FL, DL, Hex, Kz,D= smp.symbols('Hx Hy Hz a g H_{FL} H_{DL} H_{ex} K_{z} D ', real=True)
m1x, m1y, m1z, m2x, m2y, m2z = smp.symbols(r'm_{1x} m_{1y} m_{1z} m_{2x} m_{2y} m_{2z}', cls=smp.Function)
# Magnetic Moments
m1x = m1x(t)
m1y = m1y(t)
m1z = m1z(t)
m2x = m2x(t)
m2y = m2y(t)
m2z = m2z(t)
# Derivatives
m1x_d = smp.diff(m1x,t)
m1y_d = smp.diff(m1y,t)
m1z_d = smp.diff(m1z,t)
m2x_d = smp.diff(m2x,t)
m2y_d = smp.diff(m2y,t)
m2z_d = smp.diff(m2z,t)

m1=Matrix([m1x,m1y,m1z])
m2=Matrix([m2x,m2y,m2z])

m1_d=Matrix([m1x_d,m1y_d,m1z_d])
m2_d=Matrix([m2x_d,m2y_d,m2z_d])

X=Matrix([1,0,0])
Y=Matrix([0,-1,0])
Z=Matrix([0,0,1])

H=Matrix([Hx,Hy,Hz])
THETA=0/180*np.pi
PHI=135/180*np.pi
e=Matrix([np.sin(THETA)*np.cos(PHI),np.sin(THETA)*np.sin(PHI),np.cos(THETA)])

Setting the free energy and calculating effective fields:

In [3]:
FE= Hex*m1.dot(m2) + Kz*((e.dot(m1))**2+(e.dot(m2))**2) - D*e.dot(m1.cross(m2)) - H.dot(m1+m2)
Heff1=smp.diff(FE,m1)
Heff2=smp.diff(FE,m2)

Now we define the 2-coupled LLG equations for each sublattice moment:

In [4]:
llg1=-m1_d + g*(m1.cross(Heff1))+a*(m1.cross(m1_d))+g*FL*Y.cross(m1)+g*DL*m1.cross(Y.cross(m1))
llg2=-m2_d + g*(m2.cross(Heff2))+a*(m2.cross(m2_d))+g*FL*Y.cross(m2)+g*DL*m2.cross(Y.cross(m2))

We solve symbollically both equations for $\frac{d\vec{m}_{1,2}}{dt}$: (we don't solve the differential equation yet. We just have a symbollic expression for $\frac{d\vec{m}_{1,2}}{dt}$)

In [5]:
sols12=smp.solve(Matrix([llg1[0],llg1[1],llg1[2],llg2[0],llg2[1],llg2[2],]), (m1x_d, m1y_d, m1z_d,m2x_d, m2y_d, m2z_d),
                simplify=False, rational=False)

Then we set the values of material parameters such as exchange, Gilbert damping, DMI and anisotropy:

In [6]:
a_value=0.01
g_value=1
D_value=2
Hex_value=900
Kz_value=0.01
g_value=1
DL_value=0.001
FL_value=0.001
Hx_value,Hy_value,Hz_value=0,3,3
t = smp.symbols('t', real=True)
Hx, Hy, Hz, a, g, FL, DL, Hex, Kz,D= smp.symbols('Hx Hy Hz a g H_{FL} H_{DL} H_{ex} K_{z} D ', real=True)
m1x, m1y, m1z, m2x, m2y, m2z = smp.symbols(r'm_{1x} m_{1y} m_{1z} m_{2x} m_{2y} m_{2z}', cls=smp.Function)
# Magnetic Moments
m1x = m1x(t)
m1y = m1y(t)
m1z = m1z(t)
m2x = m2x(t)
m2y = m2y(t)
m2z = m2z(t)
# Derivatives
m1x_d = smp.diff(m1x,t)
m1y_d = smp.diff(m1y,t)
m1z_d = smp.diff(m1z,t)
m2x_d = smp.diff(m2x,t)
m2y_d = smp.diff(m2y,t)
m2z_d = smp.diff(m2z,t)

m1=Matrix([m1x,m1y,m1z])
m2=Matrix([m2x,m2y,m2z])

m1_d=Matrix([m1x_d,m1y_d,m1z_d])
m2_d=Matrix([m2x_d,m2y_d,m2z_d])

X=Matrix([1,0,0])
Y=Matrix([0,-1,0])
Z=Matrix([0,0,1])

H=Matrix([Hx,Hy,Hz])


We evaluate the constants:

In [7]:
m1x_df = smp.lambdify((t,m1,m2), sols12[m1x_d].subs([(Hex,Hex_value),(g,g_value),(a,a_value),(D,D_value),(Kz,Kz_value),(FL,FL_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))
m1y_df = smp.lambdify((t,m1,m2), sols12[m1y_d].subs([(Hex,Hex_value),(g,g_value),(a,a_value),(D,D_value),(Kz,Kz_value),(FL,FL_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))
m1z_df = smp.lambdify((t,m1,m2), sols12[m1z_d].subs([(Hex,Hex_value),(g,g_value),(a,a_value),(D,D_value),(Kz,Kz_value),(FL,FL_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))
m2x_df = smp.lambdify((t,m1,m2), sols12[m2x_d].subs([(Hex,Hex_value),(g,g_value),(a,a_value),(D,D_value),(Kz,Kz_value),(FL,FL_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))
m2y_df = smp.lambdify((t,m1,m2), sols12[m2y_d].subs([(Hex,Hex_value),(g,g_value),(a,a_value),(D,D_value),(Kz,Kz_value),(FL,FL_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))
m2z_df = smp.lambdify((t,m1,m2), sols12[m2z_d].subs([(Hex,Hex_value),(g,g_value),(a,a_value),(D,D_value),(Kz,Kz_value),(FL,FL_value),(DL,DL_value),(Hx,Hx_value),(Hy,Hy_value),(Hz,Hz_value)]))

We define a function and inital conditions so that we can solve the ODES numerically:

In [8]:
def dSdt(t,S):
    m1x,m1y,m1z,m2x,m2y,m2z  = S
    return [
        m1x_df(t,[m1x,m1y,m1z],[m2x,m2y,m2z]),
        m1y_df(t,[m1x,m1y,m1z],[m2x,m2y,m2z]),
        m1z_df(t,[m1x,m1y,m1z],[m2x,m2y,m2z]),
        m2x_df(t,[m1x,m1y,m1z],[m2x,m2y,m2z]),
        m2y_df(t,[m1x,m1y,m1z],[m2x,m2y,m2z]),
        m2z_df(t,[m1x,m1y,m1z],[m2x,m2y,m2z]),
    ]

theta1=75*(np.pi/180)
phi1=45*(np.pi/180)

x1=np.sin(theta1)*np.cos(phi1)
y1=np.sin(theta1)*np.sin(phi1)
z1=np.cos(theta1)

theta2=90*(np.pi/180)
phi2=135*(np.pi/180)

x2=np.sin(theta2)*np.cos(phi2)
y2=np.sin(theta2)*np.sin(phi2)
z2=np.cos(theta2)

init_cond=(x1,y1,z1,-x1,-y1,-z1)

We solve the coupled ODEs using ODEINT function:

In [9]:
t = np.linspace(0, 50, 1200)
number_of_points=len(t)
sol1 = odeint(dSdt, y0=init_cond, t=t, tfirst=True)
sol=sol1.T
#sol2 = solve_ivp(dSdt, t_span=(0,max(t)), y0=init_cond, t_eval=t).y
#sol=sol2

We plot the graphics:

In [14]:
X1s=sol[0]
Y1s=sol[1]
Z1s=sol[2]
X2s=sol[3]
Y2s=sol[4]
Z2s=sol[5]
fig1 = go.FigureWidget()
#NEEL VECTOR1
x1 = np.array([0,X1s[0]])
y1 = np.array([0,Y1s[0]])
z1 = np.array([0,Z1s[0]])

fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.75,
    marker=dict(size=0.1,color='black'),
    line=dict(color='red', width=4 ),
    name="$m_{1}$")
fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
    sizeref=0.1,colorscale=[[0, 'red'], [1,'red']])


#NEEL VECTOR2
x2 = np.array([0,X2s[0]])
y2 = np.array([0,Y2s[0]])
z2 = np.array([0,Z2s[0]])
fig1.add_scatter3d(x=x2, y=y2, z=z2,opacity=0.75,
    marker=dict(size=0.1,color='black'),
    line=dict(color='blue', width=4 ),
    name="$m_{2}$")
fig1.add_cone(x=[x2[1]], y=[y2[1]], z=z2[[1]], u=[x2[1]+x2[1]], v=[y2[1]+y2[1]], w=[z2[1]+z2[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
    sizeref=0.1,colorscale=[[0, 'blue'], [1,'blue']])


##Torques on Neel Vector1
Y=[0,-1,0]
m1=[X1s[0],Y1s[0],Z1s[0]]
FL=np.cross(Y,m1)
FL=FL/(5*np.linalg.norm(FL))
x1 = np.array([X1s[0],X1s[0]+FL[0]])
y1 = np.array([Y1s[0],Y1s[0]+FL[1]])
z1 = np.array([Z1s[0],Z1s[0]+FL[2]])

#Field-like Torque
fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.9,
    marker=dict(size=0.1,color='black'),
    line=dict(color='orange', width=5 ),
    name="$\\tau_{FL}$",showlegend=True)
fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]-x1[0]], v=[y1[1]-y1[0]], w=[z1[1]-z1[0]],opacity=0.75,showscale=False,anchor="tail",sizemode="absolute",
    sizeref=0.05,colorscale=[[0, 'orange'], [1,'orange']])

m1=[X1s[0],Y1s[0],Z1s[0]]
DL=np.cross(m1,np.cross(Y,m1))
DL=DL/(5*np.linalg.norm(DL))
x1 = np.array([X1s[0],X1s[0]+DL[0]])
y1 = np.array([Y1s[0],Y1s[0]+DL[1]])
z1 = np.array([Z1s[0],Z1s[0]+DL[2]])

#Damping-like Torque
fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.9,
    marker=dict(size=0.1,color='black'),
    line=dict(color='brown', width=5 ),
    name="$ \\tau_{DL}$",showlegend=True)
fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]],  u=[x1[1]-x1[0]], v=[y1[1]-y1[0]], w=[z1[1]-z1[0]],opacity=0.75,showscale=False,anchor="tail",sizemode="absolute",
    sizeref=0.05,colorscale=[[0, 'brown'], [1,'brown']])


##Torques on Neel Vector2

m2=[X2s[0],Y2s[0],Z2s[0]]
FL=np.cross(Y,m2)
FL=FL/(5*np.linalg.norm(FL))
x1 = np.array([X2s[0],X2s[0]+FL[0]])
y1 = np.array([Y2s[0],Y2s[0]+FL[1]])
z1 = np.array([Z2s[0],Z2s[0]+FL[2]])

#Field-like Torque
fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.9,
    marker=dict(size=0.1,color='black'),
    line=dict(color='orange', width=5 ),
    name="$\\tau_{FL}$",showlegend=False)
fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]-x1[0]], v=[y1[1]-y1[0]], w=[z1[1]-z1[0]],opacity=0.75,showscale=False,anchor="tail",sizemode="absolute",
    sizeref=0.05,colorscale=[[0, 'orange'], [1,'orange']])


m2=[X2s[0],Y2s[0],Z2s[0]]
DL=np.cross(m2,np.cross(Y,m2))
DL=DL/(5*np.linalg.norm(DL))
x1 = np.array([X2s[0],X2s[0]+DL[0]])
y1 = np.array([Y2s[0],Y2s[0]+DL[1]])
z1 = np.array([Z2s[0],Z2s[0]+DL[2]])

#Damping-like Torque
fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.9,
    marker=dict(size=0.1,color='black'),
    line=dict(color='brown', width=5 ),
    name="$ \\tau_{DL}$",showlegend=False)
fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]],  u=[x1[1]-x1[0]], v=[y1[1]-y1[0]], w=[z1[1]-z1[0]],opacity=0.75,showscale=False,anchor="tail",sizemode="absolute",
    sizeref=0.05,colorscale=[[0, 'brown'], [1,'brown']])

##excessmoment
factor=100
x2 = factor*np.array([0,X1s[0]+X2s[0]])
y2 = factor*np.array([0,Y1s[0]+Y2s[0]])
z2 = factor*np.array([0,Z1s[0]+Z2s[0]])
fig1.add_scatter3d(x=x2, y=y2, z=z2,opacity=0.75,
   marker=dict(size=0.1,color='black'),
   line=dict(color='black', width=5 ),
   name="$M$")

m1=[X1s[0],Y1s[0],Z1s[0]]
m2=[X2s[0],Y2s[0],Z2s[0]]
m1=np.asfarray(m1)
m2=np.asfarray(m2)
np.cross(m1,-1*m2)
#exchange-Torque1
ex1=1e3*np.cross(m1,-1*m2)
x1 = np.array([X2s[0],X2s[0]+ex1[0]])
y1 = np.array([Y2s[0],Y2s[0]+ex1[1]])
z1 = np.array([Z2s[0],Z2s[0]+ex1[2]])
fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=1,
   marker=dict(size=0.1,color='black'),
   line=dict(color='purple', width=7 ),
   name="$H_EX1$")



fig1.update_layout(
    height=700,
    width=900,

    autosize=True,
    scene=dict(camera=dict(up=dict(x=0,y=0, z=1 ), eye=dict(x=-1,y=1,z=1,)),
               xaxis = dict(nticks=6, range=[-2,2]),
               yaxis = dict(nticks=6, range=[-2,2]),
               zaxis = dict(nticks=6, range=[-2,2]),
        aspectratio = dict( x=1, y=1, z=1),
        aspectmode = 'manual'))


theta, phi = np.mgrid[0:2*np.pi:100j, 0:2*np.pi:100j]
x = np.sin(theta)*np.sin(phi)
y = np.sin(theta)*np.cos(phi)
z = np.cos(theta)

fig1.add_surface(x=x,y=y,z=z,opacity=0.07,colorscale=[[0, 'grey'], [1,'white']],showlegend=False, showscale = False)

fig1.add_scatter3d(x=X1s, y=Y1s, z=Z1s,opacity=0.25,
    marker=dict(size=0.1,color='red'),
    line=dict(color='red', width=3),
    showlegend=False)

fig1.add_scatter3d(x=X2s, y=Y2s, z=Z2s,opacity=0.25,
    marker=dict(size=0.1,color='blue'),
    line=dict(color='blue', width=3 ),
    showlegend=False)

#Extermal Field1
norm=2*np.sqrt(float(Hx_value**2+Hy_value**2+Hz_value**2))
x1 = np.array([0,Hx_value/norm])
y1 = np.array([0,Hy_value/norm])
z1 = np.array([0,Hz_value/norm])

fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.75,
    marker=dict(size=0.1,color='black'),
    line=dict(color='green', width=6 ),
    name="$H_{applied}$")
fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
    sizeref=0.1,colorscale=[[0, 'green'], [1,'green']])

#Spin Accumulation

x1 = np.array([0,0])
y1 = np.array([0,0.5])
z1 = np.array([0,0])

fig1.add_scatter3d(x=x1, y=y1, z=z1,opacity=0.75,
    marker=dict(size=0.1,color='black'),
    line=dict(color='yellow', width=6),
    name="$\sigma_{AC}$")
fig1.add_cone(x=[x1[1]], y=[y1[1]], z=z1[[1]], u=[x1[1]+x1[1]], v=[y1[1]+y1[1]], w=[z1[1]+z1[1]],opacity=0.75,showscale=False,anchor="tip",sizemode="absolute",
    sizeref=0.1,colorscale=[[0, 'yellow'], [1,'yellow']])

#CIRCLE IN XY PLANE
C1=1
phi=np.linspace(0,2*np.pi,100)
x=C1*np.cos(phi)
y=C1*np.sin(phi)
z=C1*np.zeros(len(x))
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.2,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

#CIRCLE IN XZ PLANE
phi=np.linspace(0,2*np.pi,100)
x=C1*np.cos(phi)
y=C1*np.zeros(len(x))
z=C1*np.sin(phi)
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.2,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

#CIRCLE IN YZ PLANE
phi=np.linspace(0,2*np.pi,100)
x=C1*np.zeros(len(x))
y=C1*np.cos(phi)
z=C1*np.sin(phi)
fig1.add_scatter3d(x=x, y=y, z=z,opacity=0.2,
    marker=dict(size=0.1,color='grey'),
    line=dict(color='grey', width=2 ),
    showlegend=False)

scaleTorques=0.5
def update(t=0):
    with fig1.batch_update():
        #Neel Vector1
        fig1.data[0].x=[0,X1s[t]]
        fig1.data[0].y=[0,Y1s[t]]
        fig1.data[0].z=[0,Z1s[t]]
        #Neel Vector1's Cone
        fig1.data[1].x=[X1s[t]]
        fig1.data[1].y=[Y1s[t]]
        fig1.data[1].z=[Z1s[t]]
        fig1.data[1].u=[2*X1s[t]]
        fig1.data[1].v=[2*Y1s[t]]
        fig1.data[1].w=[2*Z1s[t]]
        #Neel Vector2
        fig1.data[2].x=[0,X2s[t]]
        fig1.data[2].y=[0,Y2s[t]]
        fig1.data[2].z=[0,Z2s[t]]
        #Neel Vector2's Cone
        fig1.data[3].x=[X2s[t]]
        fig1.data[3].y=[Y2s[t]]
        fig1.data[3].z=[Z2s[t]]
        fig1.data[3].u=[2*X2s[t]]
        fig1.data[3].v=[2*Y2s[t]]
        fig1.data[3].w=[2*Z2s[t]]
        
        #Torques on Neel Vector1
        Y=[0,-1,0]
        m1=np.asfarray([X1s[t],Y1s[t],Z1s[t]])
        FL1=np.cross(Y,m1)*scaleTorques
        #FL=FL/(5*np.linalg.norm(FL))
        x1 = np.array([X1s[t],X1s[t]+FL1[0]])
        y1 = np.array([Y1s[t],Y1s[t]+FL1[1]])
        z1 = np.array([Z1s[t],Z1s[t]+FL1[2]])
        #FL Torque1
        fig1.data[4].x=x1
        fig1.data[4].y=y1
        fig1.data[4].z=z1
        #FL Torque1's Cone
        fig1.data[5].x=[x1[1]]
        fig1.data[5].y=[y1[1]]
        fig1.data[5].z=[z1[1]]
        fig1.data[5].u=[x1[1]-x1[0]]
        fig1.data[5].v=[y1[1]-y1[0]]
        fig1.data[5].w=[z1[1]-z1[0]] 
        
        #DL Torque1
        DL1=np.cross(m1,np.cross(Y,m1))*scaleTorques
        #DL=DL/(5*np.linalg.norm(DL))
        x1 = np.array([X1s[t],X1s[t]+DL1[0]])
        y1 = np.array([Y1s[t],Y1s[t]+DL1[1]])
        z1 = np.array([Z1s[t],Z1s[t]+DL1[2]])
        
        #DL Torque1
        fig1.data[6].x=x1
        fig1.data[6].y=y1
        fig1.data[6].z=z1
        #DL Torque1'cone
        fig1.data[7].x=[x1[1]]
        fig1.data[7].y=[y1[1]]
        fig1.data[7].z=[z1[1]]
        fig1.data[7].u=[x1[1]-x1[0]]
        fig1.data[7].v=[y1[1]-y1[0]]
        fig1.data[7].w=[z1[1]-z1[0]]
        
        #FL Torque2
        m2=np.asfarray([X2s[t],Y2s[t],Z2s[t]])
        FL2=np.cross(Y,m2)*scaleTorques
        x1 = np.array([X2s[t],X2s[t]+FL2[0]])
        y1 = np.array([Y2s[t],Y2s[t]+FL2[1]])
        z1 = np.array([Z2s[t],Z2s[t]+FL2[2]])
        
        fig1.data[8].x=x1
        fig1.data[8].y=y1
        fig1.data[8].z=z1
        
        fig1.data[9].x=[x1[1]]
        fig1.data[9].y=[y1[1]]
        fig1.data[9].z=[z1[1]]
        fig1.data[9].u=[x1[1]-x1[0]]
        fig1.data[9].v=[y1[1]-y1[0]]
        fig1.data[9].w=[z1[1]-z1[0]] 
        
        #DL Torque2
        DL2=np.cross(m2,np.cross(Y,m2))*scaleTorques        
        x1 = np.array([X2s[t],X2s[t]+DL2[0]])
        y1 = np.array([Y2s[t],Y2s[t]+DL2[1]])
        z1 = np.array([Z2s[t],Z2s[t]+DL2[2]])

        fig1.data[10].x=x1
        fig1.data[10].y=y1
        fig1.data[10].z=z1
        
        
        fig1.data[11].x=[x1[1]]
        fig1.data[11].y=[y1[1]]
        fig1.data[11].z=[z1[1]]
        fig1.data[11].u=[x1[1]-x1[0]]
        fig1.data[11].v=[y1[1]-y1[0]]
        fig1.data[11].w=[z1[1]-z1[0]] 
        
#         #Excess Moment
#         factor=10
#         fig1.data[12].x=factor*np.array([0,X1s[t]+X2s[t]])
#         fig1.data[12].y=factor*np.array([0,Y1s[t]+Y2s[t]])
#         fig1.data[12].z=factor*np.array([0,Z1s[t]+Z2s[t]]) 
#         m1=np.asfarray(m1)


#         #Exchange Torque1
#         ex1=100*np.cross(m1,-1*m2)
#         fig1.data[13].x=[X1s[t],X1s[t]+ex1[0]]
#         fig1.data[13].y=[Y1s[t],Y1s[t]+ex1[1]]
#         fig1.data[13].z=[Z1s[t],Z1s[t]+ex1[2]]
w=interact(update,t=widgets.IntSlider(min=0, max=number_of_points-1, step=int(number_of_points)/25, value=0,description="Time Step",readout_format=".1d",continuous_update=False))
#w=interactive(update,t=(0,number_of_points-1,1)) 
#w.children[0].description="$Time (t)$"
#display(w)
fig1



interactive(children=(IntSlider(value=0, continuous_update=False, description='Time Step', max=1199, readout_f…

FigureWidget({
    'data': [{'line': {'color': 'red', 'width': 4},
              'marker': {'color': 'black', …